In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import ast
from together import Together
from alive_progress import alive_bar
from collections import Counter

from prompt import *
from classes.utils import extract_option

PATH = "/home/cc/PHD/HealthBranches/"

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/dataset_updated.csv")

In [ ]:
i = 0
for c in df['correct_option'].values:
    if c not in ["A", "B", "C", "D", "E"]:
        print(i)
        print(df.loc[i]['correct_option'])
    i = i+1

In [ ]:
df = pd.read_csv(f"{PATH}results/results_quiz_Llama-3.3-70B-Instruct-Turbo-Free.csv")
df["zero_shot_new"] = df["zero_shot"].apply(extract_option)

big_filtered = df[df["zero_shot_new"].str.upper() != df["real"].str.upper()]
len(big_filtered)

In [ ]:
small_df = pd.read_csv(f"{PATH}results/results_quiz_mistral.csv")
small_df = small_df[small_df["question"].isin(big_filtered["question"])]

In [ ]:
small_df["zero_shot_new"] = small_df["zero_shot"].apply(extract_option)
small_filtered = small_df[small_df["zero_shot_new"].str.upper() != small_df["real"].str.upper()]

In [ ]:
# Directory contenente i file CSV
csv_directory = "/home/cc/PHD/HealthBranches/quiz-ultimate-dataset"

# Leggere tutti i CSV
csv_files = [os.path.join(csv_directory, f) for f in os.listdir(csv_directory) if f.endswith(".csv")]
questions = []

print(csv_files)

# Elaborazione di ogni CSV
for file in csv_files:
    df = pd.read_csv(file)

    # Rimuove la colonna "zero_shot_rag" se esiste
    if "zero_shot_rag" in df.columns:
        df = df.drop(columns=["zero_shot_rag"])

    # Applica la funzione di estrazione su zero_shot
    df["zero_shot"] = df["zero_shot"].apply(extract_option)

    # Filtra le righe in cui zero_shot è diverso da real
    df_filtered = df[df["zero_shot"] != df["real"]].copy()

    questions.extend(df_filtered['question'].values)

In [ ]:
# Conta le occorrenze
conteggio = Counter(questions)

# Ordina in ordine decrescente
dizionario_ordinato = dict(sorted(conteggio.items(), key=lambda x: x[1], reverse=True))

filtered_dict = {k: v for k, v in dizionario_ordinato.items() if v >= 6}

print(len(filtered_dict))

In [ ]:
questions_to_check = pd.read_csv("/home/cc/PHD/HealthBranches/questions_to_check.csv")

# Trova le domande che sono sia nel dizionario che nel DataFrame
matching_questions = [q for q in filtered_dict.keys() if q in questions_to_check["question"].values]
print(len(matching_questions))

print("Domande trovate nel DataFrame:", len(matching_questions))

In [ ]:
# Carica il CSV (sostituisci 'file.csv' con il tuo file)
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_checker_70B.csv")

# Conta le occorrenze di ogni valore nella colonna "check"
conteggio = df["check"].value_counts().sort_index()

# Stampa il risultato
print(conteggio)

In [ ]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain.prompts import ChatPromptTemplate
from typing import List

class LLMinference:
    def __init__(self, llm_name, temperature=0.0, num_predict=128):
        self.llm_name = llm_name
        self.model = OllamaLLM(model=llm_name, temperature=temperature, num_predict=num_predict,
                               system="Answer the question directly without including any reasoning or explanation. Do not use <think> tags or any analytical text.") 

    def _transform_query(self, query: str) -> str:
        return f'Represent this sentence for searching relevant passages: {query}'

    def single_inference(self, query: str, template: str, path: str, text: str,  choices: List[str], cond: str,  context) -> str | List[str]:
        context_text = "\n\n---\n\n".join([doc.page_content for doc in context])
        prompt_template = ChatPromptTemplate.from_template(template)
        
        if choices: # quiz
            if path != "" and text != "":
                prompt = prompt_template.format(context=context_text, question=query, path=path, text=text, condition=cond, o1=choices[0], o2=choices[1], o3=choices[2], o4=choices[3], o5=choices[4])
            else:
                prompt = prompt_template.format(context=context_text, question=query, condition=cond, o1=choices[0], o2=choices[1], o3=choices[2], o4=choices[3], o5=choices[4])
        else: # open question
            if path != "" and text != "":
                prompt = prompt_template.format(context=context_text, question=query, path=path, text=text, condition=cond)
            else:
                prompt = prompt_template.format(context=context_text, question=query, condition=cond)

        response_text = self.model.invoke(prompt)
        response_text = response_text.strip().replace("\n", "").replace("  ", "")

        sources = [doc.metadata.get("source", None) for doc in context]
        
        return response_text, sources

    def qea_evaluation(self, query: str, template: str, path: str, txt: str, choices: List[str], cond: str, vector_store, k: int = 3) -> str:

        results = vector_store.search(query=query, k=3)

        response, sources = self.single_inference(query, template, path, txt, choices, cond, results)

        return response

    def inf(self, string):
        response_text = self.model.invoke(string)
        return response_text

In [ ]:
deep = LLMinference("deepseek-r1:8b")

In [ ]:
deep.inf("Question: what is the capital of germany? \n Answer: ")

In [1]:
import pandas as pd
import ast
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# Se la GPU è disponibile, usa 'cuda', altrimenti 'cpu'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Esecuzione su: {device}")

# Carica il modello sulla GPU se disponibile
model = SentenceTransformer('all-mpnet-base-v2', device=device)

# Sostituisci 'data.csv' con il percorso del tuo file CSV.
df = pd.read_csv('/home/cc/PHD/HealthBranches/questions_pro/dataset_updated_V2path.csv')

# Definisce la soglia di similarità (80%)
SIMILARITY_THRESHOLD = 0.9

# Pre-elabora il DataFrame:
# 1. Dividi il campo 'path' in una lista di segmenti
df['path_segments'] = df['path'].apply(lambda x: [seg.strip() for seg in x.split("->")])
# 2. Estrai il testo dell'opzione corretta
def extract_correct_option(row):
    try:
        options_list = ast.literal_eval(row['options'].replace("['", '["').replace("']", '"]').replace("', '", '", "'))
    except (ValueError, SyntaxError) as e:
        # Se il parsing fallisce, restituisce una stringa vuota
        return ""
    correct_letter = row['correct_option'].strip().upper()
    index = ord(correct_letter) - ord('A')
    if index < 0 or index >= len(options_list):
        return ""
    return options_list[index]

df['correct_option_text'] = df.apply(extract_correct_option, axis=1)

# Elabora in batch gli embedding delle opzioni corrette
correct_texts = df['correct_option_text'].tolist()
correct_embeddings = model.encode(correct_texts, device=device, convert_to_tensor=True)

# Funzione per verificare in una riga se almeno un segmento del path è semanticamente simile all'opzione corretta
def check_similar_option_batch(idx, row):
    segments = row['path_segments']
    # Se non ci sono segmenti, ritorna False
    if not segments:
        return False
    # Calcola in batch gli embedding dei segmenti (convert_to_tensor=True per operare su GPU)
    segments_embeddings = model.encode(segments, device=device, convert_to_tensor=True)
    # Recupera l'embedding dell'opzione corretta per la riga corrente
    correct_emb = correct_embeddings[idx]
    # Normalizza gli embedding per applicare correttamente il calcolo della similarità coseno
    correct_norm = correct_emb / correct_emb.norm()
    segments_norm = segments_embeddings / segments_embeddings.norm(dim=1, keepdim=True)
    # Calcola la similarità coseno tra l'opzione e ogni segmento
    similarities = torch.matmul(segments_norm, correct_norm)
    # Se almeno una similarità supera la soglia, ritorna True
    return (similarities >= SIMILARITY_THRESHOLD).any().item()

# Processa ogni riga e determina se il match semantico supera la soglia
results = [check_similar_option_batch(idx, row) for idx, row in tqdm(df.iterrows())]
df['is_similar_in_path'] = results

# Calcola e stampa il contatore finale
counter = sum(results)
print(f"Numero di righe in cui almeno un segmento del path risulta simile (>= 80%) all'opzione corretta: {counter}")

# Visualizza il DataFrame aggiornato
print(df)

Esecuzione su: cuda


0it [00:00, ?it/s]

Numero di righe in cui almeno un segmento del path risulta simile (>= 80%) all'opzione corretta: 839
      index                                           question  \
0         0  A 72-year-old male presents with symptoms of P...   
1         1  A 70-year-old patient presents with resting tr...   
2         2  A 68-year-old male presents with a resting tre...   
3         4  A 72-year-old patient presents with resting tr...   
4         5  A 68-year-old male presents with a resting tre...   
...     ...                                                ...   
4105   4109  A 55-year-old male presents with difficulty sw...   
4106   4110  A 58-year-old male presents with difficulty sw...   
4107   4111  A 55-year-old male presents to your office com...   
4108   4112  A 55-year-old male presents with a year-long h...   
4109   4113  A 52-year-old male with a history of bipolar d...   

                                                options correct_option  \
0     ['Start a dopamine agonist